# ToDo
- [x] Session-parallel mini-batch generation, where each session lengths is tracked
- [ ] (Optional) Add input connections to multiple GRU layers
- [x] Make the Loss Function Compatible with the Session-parallel mini-batch generation step
- [x] Output Sampling
- [x] Test Code(making predictions)
- [x] Check Loss Function
- [x] Check Optimizer
- [x] Check GRU layer
- [x] Check Evaluation code
- [ ] Simplify the code: Debug `run_train_simple.py`, `generator.py`, `model_simple.py`

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

PATH_HOME = Path.home()
PATH_PROJ = PATH_HOME/'pyGRU4REC' 
PATH_DATA = PATH_PROJ/'data'
train = 'rsc15_train_full.txt'
test = 'rsc15_test.txt'
PATH_TO_TRAIN = PATH_DATA / train
PATH_TO_TEST = PATH_DATA / test

df_train = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
df_test = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

# Train

In [2]:
from modules.model import GRU4REC
import torch

session_key = 'SessionId'
time_key = 'Time'
item_key = 'ItemId'

input_size = df_train[item_key].nunique()
hidden_size = 100
num_layers = 1
output_size = input_size
batch_size = 50
dropout_input = 0
dropout_hidden = .5

optimizer_type = 'Adagrad'
lr = .01
weight_decay = 0
momentum = 0
eps = 1e-6

loss_type = 'TOP1'

n_epochs = 10
time_sort = False
# n_samples = 10000
n_samples = -1
use_cuda = True

torch.manual_seed(7)
torch.cuda.manual_seed(7)

train_data = df_train[:n_samples] if n_samples != -1 else df_train

model = GRU4REC(input_size, hidden_size, output_size, num_layers=num_layers, batch_size=batch_size,
                dropout_input=dropout_input, dropout_hidden=dropout_hidden, optimizer_type=optimizer_type, lr=lr,
                weight_decay=weight_decay, momentum=momentum, eps=eps, loss_type=loss_type,
                time_sort=time_sort, use_cuda=use_cuda)

model.train(train_data, session_key, time_key, item_key, n_epochs=n_epochs)

# Evaluate

## Load the data and related modules

In [1]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
from modules.layer import GRU
from modules.model import GRU4REC
import torch

model_root = Path('./models')
dataset_root = Path('../sess-rec-large')
train = 'rsc15_train_full.txt'
test = 'rsc15_test.txt'
PATH_TO_TRAIN = dataset_root / train
PATH_TO_TEST = dataset_root / test

session_key = 'SessionId'
time_key = 'Time'
item_key = 'ItemId'

df_train = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
df_test = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

## Load the common parameters

In [2]:
input_size = df_train[item_key].nunique()
output_size = input_size
hidden_size = 100
num_layers = 1

use_cuda = True
time_sort = False

optimizer_type = 'Adagrad'
dropout_input = 0

### 1. Evaluation with TOP1 Loss

In [4]:
model_name = 'GRU4REC_TOP1_Adagrad_0.01_epoch10'
model_file = model_root/model_name

loss_type = 'TOP1'
lr = 0.01

dropout_hidden = .5
dropout_input = 0
batch_size = 50
momentum = 0

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = dropout_input,
                dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = loss_type, optimizer_type = optimizer_type,
                lr=lr,
                momentum=momentum,
                time_sort=time_sort,
                pretrained=gru)

k = 20
recall, mrr = model.test(df_train, df_test, session_key,
                         time_key, item_key,
                         k = k, batch_size = batch_size)
result = f'Recall@{k}:{recall:.7f},MRR@{k}:{mrr:.7f}'
print(result)

Switching into Testing mode.
Switching into Training mode.
Recall@20:0.6171223,MRR@20:0.2565030


In [5]:
model_name = 'GRU4REC_BPR_Adagrad_0.05_epoch10'
model_file = model_root/model_name

loss_type = 'BPR'
lr = 0.05

dropout_hidden = .2
dropout_input = 0
batch_size = 50
momentum = 0.2

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = dropout_input,
                dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = loss_type, optimizer_type = optimizer_type,
                lr=lr,
                momentum=momentum,
                time_sort=time_sort,
                pretrained=gru)

k = 20
recall, mrr = model.test(df_train, df_test, session_key,
                         time_key, item_key,
                         k = k, batch_size = batch_size)
result = f'Recall@{k}:{recall:.7f},MRR@{k}:{mrr:.7f}'
print(result)

Switching into Testing mode.
Switching into Training mode.
Recall@20:0.5984532,MRR@20:0.2378255


In [6]:
model_name = 'GRU4REC_CrossEntropy_Adagrad_0.01_epoch10'
model_file = model_root/model_name

loss_type = 'CrossEntropy'
lr = 0.01

dropout_hidden = 0
dropout_input = 0
batch_size = 500
momentum = 0

gru = GRU(input_size, hidden_size, output_size,
          num_layers = num_layers,
          dropout_input = dropout_input,
          dropout_hidden = dropout_hidden,
          batch_size = batch_size,
          use_cuda = use_cuda)

gru.load_state_dict(torch.load(model_file))

model = GRU4REC(input_size, hidden_size, output_size,
                num_layers = num_layers,
                dropout_input = dropout_input,
                dropout_hidden = dropout_hidden,
                batch_size = batch_size,
                use_cuda = use_cuda,
                loss_type = loss_type, optimizer_type = optimizer_type,
                lr=lr,
                momentum=momentum,
                time_sort=time_sort,
                pretrained=gru)

k = 20
recall, mrr = model.test(df_train, df_test, session_key,
                         time_key, item_key,
                         k = k, batch_size = batch_size)
result = f'Recall@{k}:{recall:.7f},MRR@{k}:{mrr:.7f}'
print(result)

Switching into Testing mode.
Switching into Training mode.
Recall@20:0.6271887,MRR@20:0.2570316
